In [1]:
"""# Importing Libraries"""
import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt
import scipy
from scipy.interpolate import interp1d
from scipy.stats import kurtosis
from scipy.stats import skew
from sklearn import preprocessing
from sklearn.preprocessing import normalize
from scipy.ndimage import zoom
from skimage.measure import block_reduce
from skimage.transform import resize
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.optimizers import Adam
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization, Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import to_categorical


2023-05-03 15:07:47.240776: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-03 15:07:47.573073: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-03 15:07:47.576997: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-03 15:07:48.966398: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
"""# Loading Data"""

path = "../../Cog_DataSets/testing/"
# path = "drive/MyDrive/Colab Notebooks/CogAge/Datasets/testing/"
test_acc = np.load(path+"testAccelerometer.npy")
test_grav = np.load(path+"testGravity.npy")
test_gyro = np.load(path+"testGyroscope.npy")
test_jinsAcc = np.load(path+"testJinsAccelerometer.npy")
test_jinsGyro = np.load(path+"testJinsGyroscope.npy")
test_Label =np.load(path+"testLabels.npy") 
test_linAcc = np.load(path+"testLinearAcceleration.npy")
test_MsAcc = np.load(path+"testMSAccelerometer.npy")
test_MsGyro = np.load(path + "testMSGyroscope.npy")
test_MsMag = np.load(path+"testMagnetometer.npy")
# test_acc

path = "../../Cog_DataSets/training/"
# path = "drive/MyDrive/Colab Notebooks/CogAge/Datasets/training/"
train_acc = np.load(path+"trainAccelerometer.npy")
train_grav = np.load(path+"trainGravity.npy")
train_gyro = np.load(path+"trainGyroscope.npy")
train_jinsAcc = np.load(path+"trainJinsAccelerometer.npy")
train_jinsGyro = np.load(path+"trainJinsGyroscope.npy")
train_Label =np.load(path+"trainLabels.npy") 
train_linAcc = np.load(path+"trainLinearAcceleration.npy")
train_MsAcc = np.load(path+"trainMSAccelerometer.npy")
train_MsGyro = np.load(path + "trainMSGyroscope.npy")
train_MsMag = np.load(path+"trainMagnetometer.npy")
# print(train_Label.shape)


In [3]:
"""# Reshape and stack Data Before Fitting to Model"""
# changing shape of sensor data to (#,400,3)
# downsample
train_acc_reshaped = block_reduce(train_acc, block_size=(1, 2, 1), func=np.mean)
train_gyro_reshaped = block_reduce(train_gyro, block_size=(1, 2, 1), func=np.mean)
train_grav_reshaped = block_reduce(train_grav, block_size=(1, 2, 1), func=np.mean)
train_linAcc_reshaped = block_reduce(train_linAcc, block_size=(1, 2, 1), func=np.mean)
# upsample
train_MsAcc_reshaped = resize(train_MsAcc, (2284, 400, 3), mode='edge')
train_MsGyro_reshaped = resize(train_MsGyro, (2284, 400, 3), mode='edge')
# upsample
train_MsMag_reshaped = np.repeat(train_MsMag, 2, axis=1)
train_jinsAcc_reshaped = np.repeat(train_jinsAcc, 5, axis=1)
train_jinsGyro_reshaped = np.repeat(train_jinsGyro, 5, axis=1)

test_acc_reshaped = block_reduce(test_acc, block_size=(1, 2, 1), func=np.mean)
test_gyro_reshaped = block_reduce(test_gyro, block_size=(1, 2, 1), func=np.mean)
test_grav_reshaped = block_reduce(test_grav, block_size=(1, 2, 1), func=np.mean)
test_linAcc_reshaped = block_reduce(test_linAcc, block_size=(1, 2, 1), func=np.mean)
test_MsAcc_reshaped = resize(test_MsAcc, (2288, 400, 3), mode='edge')
test_MsGyro_reshaped = resize(test_MsGyro, (2288, 400, 3), mode='edge')
test_MsMag_reshaped = np.repeat(test_MsMag, 2, axis=1)
test_jinsAcc_reshaped = np.repeat(test_jinsAcc, 5, axis=1)
test_jinsGyro_reshaped = np.repeat(test_jinsGyro, 5, axis=1)

In [4]:
# all data of shape #,400,3
# adding all relative data.
# Mobile training accelerometer + Mobile testing accelerometer data
train_acc_reshaped = np.append(train_acc_reshaped,test_acc_reshaped, axis=0)
train_gyro_reshaped = np.append(train_gyro_reshaped,test_gyro_reshaped, axis=0)
train_grav_reshaped = np.append(train_grav_reshaped,test_grav_reshaped, axis=0)
train_linAcc_reshaped = np.append(train_linAcc_reshaped,test_linAcc_reshaped, axis=0)
train_MsAcc_reshaped = np.append(train_MsAcc_reshaped,test_MsAcc_reshaped, axis=0)
train_MsGyro_reshaped = np.append(train_MsGyro_reshaped,test_MsGyro_reshaped, axis=0)
train_MsMag_reshaped = np.append(train_MsMag_reshaped,test_MsMag_reshaped, axis=0)
train_jinsAcc_reshaped = np.append(train_jinsAcc_reshaped,test_jinsAcc_reshaped, axis=0)
train_jinsGyro_reshaped = np.append(train_jinsGyro_reshaped,test_jinsGyro_reshaped, axis=0)


print("Shape of all sensors after up/down sample... ", train_acc_reshaped.shape, train_gyro_reshaped.shape, train_grav_reshaped.shape, train_linAcc_reshaped.shape
                       , train_MsAcc_reshaped.shape, train_MsGyro_reshaped.shape, train_MsMag_reshaped.shape,
                       train_jinsAcc_reshaped.shape, train_jinsGyro_reshaped.shape)

# all data of shape 4572,400,3
# 4572 = 2284(training) + 2288(testing)


# stack
all_data = np.stack([train_acc_reshaped, train_gyro_reshaped, train_grav_reshaped, train_linAcc_reshaped
                       , train_MsAcc_reshaped, train_MsGyro_reshaped, train_MsMag_reshaped,
                       train_jinsAcc_reshaped, train_jinsGyro_reshaped], axis=-1)

all_Label = np.append(train_Label, test_Label, axis=-1)

Shape of all sensors after up/down sample...  (4572, 400, 3) (4572, 400, 3) (4572, 400, 3) (4572, 400, 3) (4572, 400, 3) (4572, 400, 3) (4572, 400, 3) (4572, 400, 3) (4572, 400, 3)


In [5]:
# 90% training data + labels
train_data = all_data[: int(all_data.shape[0]*0.9)]
# 10% testing data + labels
test_data = all_data[int(all_data.shape[0]*0.9):]
train_labels = all_Label[: int(all_Label.shape[0]*0.9)]
test_labels = all_Label[int(all_Label.shape[0]*0.9):]

print("\nShape of training and testin data + labels...\n")
print(train_data.shape, test_data.shape)
print(train_labels.shape, test_labels.shape)


# Split the data into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(train_data, train_labels, test_size=0.2, random_state=42)

print("Shape of training + validation Data")
print(np.shape(x_train), np.shape(y_train))

y_train = to_categorical(y_train, num_classes=55)
y_val = to_categorical(y_val, num_classes=55)

print("Shape of training + validation Labels")
print(np.shape(y_train), np.shape(y_val))




Shape of training and testin data + labels...

(4114, 400, 3, 9) (458, 400, 3, 9)
(4114,) (458,)
Shape of training + validation Data
(3291, 400, 3, 9) (3291,)
Shape of training + validation Labels
(3291, 55) (823, 55)


In [13]:
# HYPER PARAMTERS

# Filter parameters, i.e. about the number of inputs processed by each neuron of the convolutional layer
filter1Size = (11,1)
filter2Size = (13,1)
filter3Size = (13,1)

# Downsampling factors of the pooling layers
poolingLayer1Factor = (2,1)
poolingLayer2Factor = (3,1)
poolingLayer3Factor = (2,1)

# Number of feature maps processed by each convolutional layer
nbFeaturesLayer1 = 50
nbFeaturesLayer2 = 40
nbFeaturesLayer3 = 30

# Activation function of the convolutional layer(s)
activationConv = 'relu'

# Output dimension of the LSTM
outputLSTM = 100

# Parameters of the dense layer
activationMLP = 'relu'
inputMLP = 500

# Training parameters
batchSize = 32
numberOfEpochs = 30
learningRate = 0.01


# Con1 Model
def normConv1(
    inputShape,
    nkerns,
    filterSizes,
    poolSizes,
    activationConv,
    inputMLP,
    activationMLP,
    nbClasses,
    withSoftmax=True):

    model = Sequential()

    model.add(BatchNormalization(input_shape=inputShape))

    # 1st convolutional + pooling + normalization layer 
    model.add(Conv2D(nkerns[0], kernel_size=filterSizes[0], activation=activationConv))
    model.add(MaxPooling2D(pool_size=poolSizes[0]))

    # Fully-connected layer
    model.add(Flatten())
    model.add(Dense(inputMLP, activation=activationMLP))

    # Softmax layer
    if withSoftmax:
        model.add(Dense(nbClasses, activation='softmax'))

    # Return the model
    return model


model = normConv1(
    inputShape=(400,3,9),
    nkerns=[nbFeaturesLayer1, nbFeaturesLayer2, nbFeaturesLayer3],
    filterSizes=[filter1Size, filter2Size, filter3Size],
    poolSizes=[poolingLayer1Factor, poolingLayer2Factor, poolingLayer3Factor],
    activationConv='relu',
    inputMLP=inputMLP,
    activationMLP=activationMLP,
    nbClasses=55,
    withSoftmax=True)

In [14]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization_2 (Batc  (None, 400, 3, 9)        36        
 hNormalization)                                                 
                                                                 
 conv2d_2 (Conv2D)           (None, 390, 3, 50)        5000      
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 195, 3, 50)       0         
 2D)                                                             
                                                                 
 flatten_2 (Flatten)         (None, 29250)             0         
                                                                 
 dense_4 (Dense)             (None, 500)               14625500  
                                                                 
 dense_5 (Dense)             (None, 55)               

In [12]:
model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(learning_rate=0.001),
    metrics=['accuracy']
)

history = model.fit(
    x_train, y_train,
    validation_data=(x_val, y_val),
    epochs=numberOfEpochs,
    batch_size=batchSize
)

Epoch 1/30
103/103 [==============================] - 7s 63ms/step - loss: 3.2906 - accuracy: 0.1799 - val_loss: 2.7147 - val_accuracy: 0.2819
Epoch 2/30
103/103 [==============================] - 7s 69ms/step - loss: 1.9581 - accuracy: 0.4765 - val_loss: 2.2324 - val_accuracy: 0.4034
Epoch 3/30
103/103 [==============================] - 7s 66ms/step - loss: 1.3503 - accuracy: 0.6114 - val_loss: 1.7518 - val_accuracy: 0.5334
Epoch 4/30
103/103 [==============================] - 7s 64ms/step - loss: 0.9474 - accuracy: 0.7250 - val_loss: 1.6697 - val_accuracy: 0.5772
Epoch 5/30
103/103 [==============================] - 7s 73ms/step - loss: 0.7174 - accuracy: 0.7852 - val_loss: 1.9846 - val_accuracy: 0.5346
Epoch 6/30
103/103 [==============================] - 7s 72ms/step - loss: 0.6263 - accuracy: 0.8089 - val_loss: 1.8405 - val_accuracy: 0.5881
Epoch 7/30
103/103 [==============================] - 7s 72ms/step - loss: 0.5155 - accuracy: 0.8362 - val_loss: 1.9203 - val_accuracy: 0.5917

KeyboardInterrupt: 

In [ ]:
# model.save('cnn1.h5')


estimatedLabels = np.argmax(model.predict(test_data),axis=-1)
estimatedLabels = estimatedLabels.flatten()

accuracy = accuracy_score(test_labels,estimatedLabels)
weightedF1 = f1_score(test_labels,estimatedLabels,average='weighted')
averageF1 = f1_score(test_labels,estimatedLabels,average='macro')
#confMat = confusion_matrix(testingLabels,estimatedLabels,labels=labelsTable.values())
allF1Scores = f1_score(test_labels,estimatedLabels,average=None)

# Print results
print('   Test accuracy = %.2f %%' % (accuracy*100))
print('   Weighted F1-score = %.4f' % (weightedF1))
print('   Average F1-score = %.4f' % (averageF1))
# print('   All F1-scores:')
# print(allF1Scores)
print('-------------------------------------------------------')